In [71]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.speedtest.net/global-index'

response = requests.get(url)

if response.status_code == 200:
    print('Successfully fetched the webpage...')

    soup = BeautifulSoup(response.content, 'html.parser')

    mobile_element = soup.find(id='column-mobileMedian')
    broadband_element = soup.find(id='column-fixedMedian')

    countries_summary_mobile = mobile_element.find(class_='countriesSummary').find_all('a')
    cities_summary_mobile = mobile_element.find(class_='citiesSummary').find_all('a')

    countries_summary_broadband = broadband_element.find(class_='countriesSummary').find_all('a')
    cities_summary_broadband = broadband_element.find(class_='citiesSummary').find_all('a')
else:
    print('Failed to fetch the webpage. Status code:', response.status_code)

Successfully fetched the webpage...


In [72]:
def fetch_data(summary, soup_id, type):
    info = []

    failed_entries = []

    for link in summary:
        res = requests.get(f"https://www.speedtest.net{link.get('href')}")

        if res.status_code == 200:
          print('Successfully fetched the webpage for ->', link.text.strip(), f"https://www.speedtest.net{link.get('href')}")

          soup = BeautifulSoup(res.content, 'html.parser')

          data = soup.find(id=soup_id)

          rank = data.find(class_='headings').find(class_='rank').find(class_='number').text.strip()
          rank_change = data.find(class_='headings').find(class_='rank').find(class_='rank-change').text.strip()

          download = data.find(class_='headings').find(class_='download').find(class_='number').text.strip()

          upload = data.find(class_='headings').find(class_='upload').find(class_='number').text.strip()

          latency = data.find(class_='headings').find(class_='latency').find(class_='number').text.strip()

          info.append({
              type: link.text.strip(),
              "position": rank, # rank named column not allowed
              "rank_change": rank_change,
              "download": download,
              "upload": upload,
              "latency": latency
          })
        else:
          failed_entries.append(link)

          print('Failed to fetch the webpage for ->', link.text.strip())

    return info, failed_entries


In [73]:
countries_mobile_data, failed_country_mobile = fetch_data(countries_summary_mobile, 'column-mobileMedian', 'country')
cities_mobile_data, failed_city_mobile = fetch_data(cities_summary_mobile, 'column-mobileCitiesMedian', 'city')

countries_broadband_data, failed_country_broadband = fetch_data(countries_summary_broadband, 'column-fixedMedian', 'country')
cities_broadband_data, failed_city_broadband = fetch_data(cities_summary_broadband, 'column-fixedCitiesMedian', 'city')

Successfully fetched the webpage for -> Qatar https://www.speedtest.net/global-index/qatar#mobile
Successfully fetched the webpage for -> United Arab Emirates https://www.speedtest.net/global-index/united-arab-emirates#mobile
Successfully fetched the webpage for -> Kuwait https://www.speedtest.net/global-index/kuwait#mobile
Successfully fetched the webpage for -> Iceland https://www.speedtest.net/global-index/iceland#mobile
Successfully fetched the webpage for -> South Korea https://www.speedtest.net/global-index/south-korea#mobile
Successfully fetched the webpage for -> Denmark https://www.speedtest.net/global-index/denmark#mobile
Successfully fetched the webpage for -> China https://www.speedtest.net/global-index/china#mobile
Successfully fetched the webpage for -> Netherlands https://www.speedtest.net/global-index/netherlands#mobile
Successfully fetched the webpage for -> United States https://www.speedtest.net/global-index/united-states#mobile
Successfully fetched the webpage for -

In [74]:
import pandas as pd

In [75]:
countries_mobile_df = pd.DataFrame(countries_mobile_data)

csv_filename = 'countries_mobile_data.csv'

countries_mobile_df.to_csv(csv_filename, index=False)

print(f"Data has been written to '{csv_filename}' successfully.")

countries_mobile_df.head()

Data has been written to 'countries_mobile_data.csv' successfully.


,country,position,rank_change,download,upload,latency
0,Qatar,1,-,313.30,27.53,19
1,United Arab Emirates,2,-,296.00,26.25,19
2,Kuwait,3,-,228.64,23.73,18
3,Iceland,4,+4,158.32,20.22,17
4,South Korea,5,-1,156.23,17.68,30


In [76]:
cities_mobile_df = pd.DataFrame(cities_mobile_data)

csv_filename = 'cities_mobile_data.csv'

cities_mobile_df.to_csv(csv_filename, index=False)

print(f"Data has been written to '{csv_filename}' successfully.")

cities_mobile_df.head()

Data has been written to 'cities_mobile_data.csv' successfully.


,city,position,rank_change,download,upload,latency
0,Ar-Rayyan,1,-,427.38,27.58,19
1,Dubai,2,-,358.00,30.81,18
2,Abu Dhabi,3,-,303.71,24.10,22
3,Doha,4,+1,287.57,28.42,19
4,Riyadh,5,+2,269.95,21.41,19


In [77]:
countries_broadband_df = pd.DataFrame(countries_broadband_data)

csv_filename = 'countries_broadband_data.csv'

countries_broadband_df.to_csv(csv_filename, index=False)

print(f"Data has been written to '{csv_filename}' successfully.")

countries_broadband_df.head()

Data has been written to 'countries_broadband_data.csv' successfully.


,country,position,rank_change,download,upload,latency
0,Singapore,1,-,284.13,229.80,4
1,United Arab Emirates,2,+3,276.02,134.85,5
2,Hong Kong (SAR),3,-,274.26,193.14,4
3,Chile,4,-2,265.12,168.13,6
4,United States,5,+1,254.01,31.37,13


In [78]:
cities_broadband_df = pd.DataFrame(cities_broadband_data)

csv_filename = 'cities_broadband_data.csv'

cities_broadband_df.to_csv(csv_filename, index=False)

print(f"Data has been written to '{csv_filename}' successfully.")

cities_broadband_df.head()

Data has been written to 'cities_broadband_data.csv' successfully.


,city,position,rank_change,download,upload,latency
0,Abu Dhabi,1,-,308.05,176.57,5
1,Beijing,2,-,291.70,39.32,12
2,Valparaíso,3,-,291.28,188.15,5
3,Lyon,4,+1,285.67,210.25,6
4,Shanghai,5,-1,285.39,59.40,6


In [83]:
# SQL CREATE TABLE statements for each DataFrame
sql_statements = []
table_data = [('countries_mobile_data', countries_mobile_df), ('cities_mobile_data', cities_mobile_df), ('countries_broadband_data', countries_broadband_df), ('cities_broadband_data', cities_broadband_df)]

database_name = 'web_scrapping'
create_db_statement = f'CREATE DATABASE IF NOT EXISTS `{database_name}`;'
use_db = f'USE `{database_name}`;'
sql_statements.append(create_db_statement)
sql_statements.append(use_db)

for df_name, df in table_data:
    columns = ', '.join([f'{col} VARCHAR(255)' for col in df.columns])
    sql_statement = f'CREATE TABLE {df_name} ({columns});'
    sql_statements.append(sql_statement)

# SQL INSERT INTO statements for each DataFrame
for df_name, df in table_data:
    insert_sql = f'INSERT INTO {df_name} ({", ".join(df.columns)}) VALUES '
    values = ', '.join([str(tuple(row)) for row in df.values])
    sql_statements.append(insert_sql + values + ';')

# SQL statements to a .sql file
sql_filename = 'database_setup.sql'
with open(sql_filename, 'w') as file:
    for sql_statement in sql_statements:
        file.write(sql_statement + '\n')

print(f"SQL statements have been written to '{sql_filename}' successfully.")


SQL statements have been written to 'database_setup.sql' successfully.
